# ************************************************************* #
#               Copyright (C) 2022 Jorge Brenes Alfaro.
#               EL5617 Trabajo Final de Graduación.
#               Escuela de Ingeniería Electrónica.
#               Tecnológico de Costa Rica.
# ************************************************************* #

This file is responsible for generating the mimetic neural network (MNN). First, the data collected from the PAHM is processed, which is reshaped as necessary for the network. Next, the model is developed using recurrent neural networks (RNN), specifically the GRU.

In [1]:
#Libraries to proccess data
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Libraries to create RNAM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dense, Dropout, GRU, TimeDistributed

import warnings
warnings.filterwarnings('ignore')

#import wandb
#from wandb.keras import WandbCallback

#wandb.login()

In [12]:
wandb.init(project="Prueba", 
           entity="mimetic-rna", 
           name='Probando',
           resume='Allow', 
           id='Probando')
wandb.config = {
    "epochs": 500,
    "batch_size": 8,
    "learning_rate":0.001,
    "window": 100,
    "Dropout": 0.35,
    "n_layers":1
}

 Process the Dataset 

In [2]:
root = '/Users/jorge/Documents/TEC/TFG/Datos_Recolectados/'
Dir = os.listdir(root)
pwm = np.array([])
angle = np.array([])

# Read all the .csv files and make an nx4 array
# Next, separate the pwm value and angle in their respective arrays.
print('******************* Process the Dataset *******************',flush=True)
print('Recolecting Data',flush=True)
for filename in Dir:
    files = pd.read_csv(root+filename)
    pwm = np.append(pwm, np.concatenate((np.zeros(100),files.values[:,2])))
    angle = np.append(angle, np.concatenate((np.zeros(100),files.values[:,3])))

X_train = []
Y_train = []
window = 100

#For each element of training set, we have "window" previous training set elements
print('Accommodating data for the GRU network',flush=True)
for i in range(window,pwm.shape[0]):
    X_train.append(pwm[i-window:i])
    Y_train.append(angle[i])
X_train, Y_train = np.array(X_train), np.array(Y_train) # Input and output arrays

# Separate the values in train, validation and test data/label
train_data, val_data, test_data = [],[],[]
train_label, val_label, test_label = [],[],[]
train_lenght = int(len(X_train)*3/5)
val_lenght = int(len(X_train)*4/5)

# Use 3/5 of the total data set for training 
# and 1/5 for validation and testing.
print('Separating data in training, validation and testing',flush=True)
for i,j in zip(X_train[:train_lenght],Y_train[:train_lenght]):
    train_data.append(i)
    train_label.append(j)

for i,j in zip(X_train[train_lenght:val_lenght],Y_train[train_lenght:val_lenght]):
    val_data.append(i)
    val_label.append(j)
    
for i,j in zip(X_train[val_lenght:],Y_train[val_lenght:]):
    test_data.append(i)
    test_label.append(j)
    
train_data, val_data, test_data = np.array(train_data), np.array(val_data), np.array(test_data)
train_label, val_label, test_label = np.array(train_label), np.array(val_label), np.array(test_label)

print('El total de datos de entrenamiento es: ', len(train_data), flush=True)
print('El total de datos de validación es: ', len(val_data), flush=True)
print('El total de datos de prueba es: ', len(test_data), flush=True)

# Reshape the arrays (n,window,1). Where n is the total amount of data in the array
print('Reshape arrays to tensors',flush=True)
train_data = np.reshape(train_data,(train_data.shape[0],train_data.shape[1],1))
val_data = np.reshape(val_data,(val_data.shape[0],val_data.shape[1],1))
test_data = np.reshape(test_data,(test_data.shape[0],test_data.shape[1],1))
print('******************* Finish *******************',flush=True)

******************* Process the Dataset *******************
Recolecting Data
Accommodating data for the GRU network
Separating data in training, validation and testing
El total de datos de entrenamiento es:  82911
El total de datos de validación es:  27637
El total de datos de prueba es:  27638
Reshape arrays to tensors
******************* Finish *******************


******************* Neural Network *******************

Model creation

In [3]:
clear_session()

model = Sequential()
model.add(GRU(units=64, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.35))
model.add(GRU(units=64, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dense(units=1))
#model.add(TimeDistributed(Dense(1))) # There is no difference between this and model.add(Dense(1))...
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse','acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 100, 64)           12864     
_________________________________________________________________
dropout (Dropout)            (None, 100, 64)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 64)           24960     
_________________________________________________________________
dense (Dense)                (None, 100, 1)            65        
Total params: 37,889
Trainable params: 37,889
Non-trainable params: 0
_________________________________________________________________


Model Training

In [4]:
history = model.fit(train_data, train_label,
                    epochs=4, batch_size=32,
                    validation_data = (val_data,val_label),
                    verbose=2)#,callbacks=[WandbCallback(save_model=False)])
# Prediction
#testPredict = model.predict(test_data)

Epoch 1/4
2591/2591 - 337s - loss: 375.4779 - mse: 375.4779 - acc: 5.2828e-04 - val_loss: 343.9978 - val_mse: 343.9978 - val_acc: 0.0000e+00
Epoch 2/4
2591/2591 - 334s - loss: 303.7562 - mse: 303.7562 - acc: 0.0213 - val_loss: 141.1970 - val_mse: 141.1970 - val_acc: 0.1148
Epoch 3/4
2591/2591 - 346s - loss: 211.9101 - mse: 211.9101 - acc: 0.0623 - val_loss: 124.1852 - val_mse: 124.1852 - val_acc: 0.0917
Epoch 4/4
2591/2591 - 353s - loss: 196.7763 - mse: 196.7763 - acc: 0.0619 - val_loss: 121.5420 - val_mse: 121.5420 - val_acc: 0.1463


Model Evaluate

In [7]:
loss,_,acc = model.evaluate(test_data,test_label,batch_size=32)

864/864 [==============================] - 26s 30ms/step - loss: 36.1743 - mse: 36.1743 - acc: 0.2636 0s - loss: 36.4551 - mse: 36.4551 - acc: 0.


26.3602614402771

In [12]:
joblib.dump(model, 'GRU_model.joblib')

TypeError: cannot pickle 'weakref' object

Plot the predicted and "true" output and plot training and validation losses

In [11]:
#Tengo que graficar real vs predicha
plt.figure()
plt.plot()
plt.plot()

loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.figure()
plt.plot(epochs, loss,'b', label='Training loss')
plt.plot(epochs, val_loss,'r', label='Validation loss')
plt.title('Training and validation losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xscale('log')
plt.legend()
plt.show()

NameError: name 'testPredict' is not defined

<Figure size 432x288 with 0 Axes>